**Necessary Libraries**

In [1]:
import torch
import pickle 
import numpy as np
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration

c:\Users\Harsh Soni\anaconda3\envs\cuda_kernels\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\Harsh Soni\anaconda3\envs\cuda_kernels\Lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary c:\Users\Harsh Soni\anaconda3\envs\cuda_kernels\Lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable


c:\Users\Harsh Soni\anaconda3\envs\cuda_kernels\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


**LLM Model and Tokenizer**

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-large")
llm_model = T5ForConditionalGeneration.from_pretrained("t5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
llm_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

**Sentence Embedding Model**

In [4]:
embed_model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\Harsh Soni\anaconda3\envs\cuda_kernels\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
embed_model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

**Loading the Document Pieces and Their Respective Embeddings**

In [6]:
with open('documents.pkl', 'rb') as f:
    documents = pickle.load(f)

embeddings = np.load('embeddings.npy')

**Utility Functions**

In [7]:
# Function to get the documnets that can most probabily contain the answer of the query

def get_similar_docs(query, top_k, docs, embeddings):
    query_embedding = embed_model.encode(query)
    similarity = embed_model.similarity(query_embedding, embeddings).squeeze(0)
    result_idx = similarity.argsort(descending=True)[:top_k]
    
    return [docs[i] for i in result_idx]

In [8]:
# Function to get the answer to the query based on the relevant documents retrieved using the function above

# Saving the history so that the model has the context to which questions are being asked.
chat_history = []

def get_answer(query):
    global chat_history

    similar_docs = get_similar_docs(query, 10, documents, embeddings)
    chat_history.append(Document(page_content=query))
    
    combined_docs = chat_history + similar_docs

    input_text = ""
    for i in combined_docs:
        input_text += i.page_content
   
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids  
    decoder_input_ids = tokenizer(query, return_tensors="pt").input_ids  
    
    outputs = llm_model.generate(input_ids=input_ids, decoder_input_ids=decoder_input_ids, max_length=500)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    chat_history.append(Document(page_content=generated_text.split(query)[1].replace('.', '. ').capitalize()))
    return generated_text.split(query)[1].capitalize()

In [9]:
query = "Describe about the Israel Hamas war."
get_answer(query)

"Israel's war with hamas that started on october 7 has now lasted for over two months.the conflict has claimed several thousands lives on both sides, with scores injured and displaced.the current conflict was triggered by the surprise hamas attack on israel on october 7, in which 1,200 israelis were gunned down in their homes, at a music festival and other public places."

In [10]:
query = "What happened at the Al-Shifa Hospital?"
get_answer(query)

'Tainment of the hospital has been a "bloodbath" hundreds of injured patients are crowded into the emergency department of al-shifa hospital, who says.'

In [11]:
query = "What has been the international response?"
get_answer(query)

'The united nations general assembly voted overwhelmingly in favor of a humanitarian ceasefire in gaza, a resolution that was supported by 153 countries, opposed by 10, and seen abstention from 23. notably, the us vetoed a similar resolution in the security council, a move that stirred controversy on the global stage.'

In [12]:
query = "What was the help provided to the victims of the war from Indian Gonvernment?"
get_answer(query)

'Naya: "india has been supporting palestine and palestinian refugees through significant contributions to unrwa between 2002 and 2023," he said. "india has been supporting palestine and palestinian refugees through significant contributions to unrwa between 2002 and 2023," he said.'